In [1]:
import os
import pandas as pd
# https://github.com/serengil/deepface
from deepface import DeepFace
import matplotlib.pyplot as plt
from PIL import Image

# https://github.com/serengil/deepface

In [14]:
#analysis = DeepFace.analyze(img_path = 'data/archive/AffectNet/Train/anger/image0021846.jpg', actions = ["age", "gender", "emotion", "race"])
#print(analysis)
# actions = ['age', 'gender', 'race', 'emotion'],

In [ ]:
def analyze_faces_in_directory(directory, emotion):
    # Create a DataFrame to store the analysis results
    results_df = pd.DataFrame(columns=["Image", "Age", "Gender", "Race", "Emotion"])

    # Loop through all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Add more extensions if needed
            file_path = os.path.join(directory, filename)
            try:
                # Analyze the face in the image with enforce_detection=False
                print(f"Processing {filename}")
                analysis = DeepFace.analyze(img_path=file_path, actions=['age', 'gender', 'race', 'emotion'], enforce_detection=False)

                # DeepFace returns a list, so access the first dictionary in the list
                analysis = analysis[0]

                # Extract data from the analysis dictionary
                age = analysis.get('age', 'N/A')  # Use .get() to handle missing keys
                gender = analysis.get('dominant_gender', 'N/A')  # Correct key for gender
                race = analysis.get('dominant_race', 'N/A')  # Correct key for race
                emotion = analysis.get('dominant_emotion', 'N/A')  # Correct key for emotion
                print(analysis)

                # Append results to the DataFrame
                results_df = results_df.append({"Image": filename, "Age": age, "Gender": gender, "Race": race, "Emotion": emotion}, ignore_index=True)

                # Print the results for the current image
                print(f"Processed {filename}: Age: {age}, Gender: {gender}, Race: {race}")

            except Exception as e:
                # Handle the error gracefully and continue processing
                print(f"Error processing {filename}: {e}")
                continue

    # Save results to CSV for further use
    results_df.to_csv(f'face_analysis_results_{e}.csv', index=False)
    print("Analysis completed! Results saved to face_analysis_results.csv.")


# Example usage
directory_path = 'datasets/AffectNet/Test'
emotion_folders = ['anger', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise', 'contempt']
for e in emotion_folders:
    analyze_faces_in_directory(f"{directory_path}/{e}", e)  


Processing image0000006.jpg


Action: age:   0%|          | 0/4 [00:00<?, ?it/s]    

24-10-17 13:08:45 - age_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5
To: C:\Users\Gergo Gyori\.deepface\weights\age_model_weights.h5


In [ ]:
Test_data = pd.read_csv('face_analysis_results.csv')
Test_data

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

def display_images_with_labels(df, image_directory):
    # Number of images to display
    num_images = len(df)
    
    # Create a 3x2 grid
    fig, axes = plt.subplots(3, 2, figsize=(8, 8))
    
    # Flatten the axes array for easier indexing
    axes = axes.flatten()
    
    for i, (index, row) in enumerate(df.iterrows()):
        if i >= 6:  # Show only 6 images max (for the 3x2 grid)
            break
        
        image_path = os.path.join(image_directory, row['Image'])
        
        # Check if the image file exists
        if os.path.exists(image_path):
            img = Image.open(image_path)
            axes[i].imshow(img)
            axes[i].axis('off')
            
            # Add label for the image
            label = f"Age: {row['Age']}\nGender: {row['Gender']}\nRace: {row['Race']}"
            axes[i].set_title(label, fontsize=14)
        else:
            # If the image doesn't exist, show a placeholder
            axes[i].text(0.5, 0.5, 'Image not found', horizontalalignment='center', verticalalignment='center', fontsize=12)
            axes[i].axis('off')
    
    # Adjust layout to make space for titles
    plt.tight_layout()
    plt.show()


# Replace with your actual image directory
image_directory = 'data/archive/AffectNet/Train/_test'

# Display the images with labels
display_images_with_labels(Test_data, image_directory)

In [ ]:
# Function to add an Age Group column
def add_age_group(df):
    # Define a function to assign age groups
    def get_age_group(age):
        lower_bound = (age // 10) * 10  # Get the lower bound of the age group
        upper_bound = lower_bound + 9  # The upper bound
        return f"{lower_bound}-{upper_bound}"

    # Apply the function to the Age column to create the Age Group column
    df['Age Group'] = df['Age'].apply(get_age_group)
    return df
df_with_age_group = add_age_group(Test_data)

df_with_age_group